In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np

In [ ]:
# Load segmentation model
modelSeg = YOLO("yolov8l-seg.pt")

In [ ]:
def calcMask(result):
    raw = result.masks[0].cpu().data.numpy().transpose(1, 2, 0)

    rgb = cv2.merge((raw, raw, raw))
    h2,w2, _ =result.orig_img.shape
    mask = cv2.resize(rgb, (w2, h2))

    mask = (mask * 255).astype(np.uint8)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    mask = cv2.GaussianBlur(mask, (5, 5), 0)

    return mask

In [ ]:
blur_ratio = 30

def generateLayer(mask, im0):
    _, binary_mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

    bg_blur = cv2.blur(im0, (blur_ratio, blur_ratio))

    inverse_mask = cv2.bitwise_not(binary_mask)

    binary_mask = cv2.resize(binary_mask, (im0.shape[1], im0.shape[0]))

    inverse_mask = cv2.resize(inverse_mask, (im0.shape[1], im0.shape[0]))

    foreground = cv2.bitwise_and(im0, im0, mask=binary_mask)
    background = cv2.bitwise_and(bg_blur, bg_blur, mask=inverse_mask)

    return foreground, background


In [ ]:
cap = cv2.VideoCapture(0)
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))


while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    results = modelSeg.predict(im0, device="mps")

    if results[0].masks is not None:
        mask = calcMask(results[0])
        foreground, background = generateLayer(mask, im0)

        combined = cv2.addWeighted(foreground, 1, background, 1, 0)

        cv2.imshow("Background blur", combined)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()